# Airbnb Data Analysis: 
## Making your house the best at the platform!
---
This notebook is part of the Udacity's Data Scientist Nanodegree program

In this project, I will be investigating Airbnb data and answering relevant questions using the **CRISP-DM** process:

1. Business Understanding
2. Data Understanding
3. Data Preparation 
4. Data Modeling
5. Results & Evaluation

# 1. Business Understanding

## The 3 questions I'm looking to answer are:

1. **Which city has the best listings? Which one has more expensive ones? Is there a connection in that?**
2. **What's the connection between price and occupation rate? Where does the quality of the listing fits in that?** 
3. **What are the main features that influences the review rates? What about the prices?**

# 2. Data Understanding

## 2.1 Importing Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import warnings
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

pd.set_option('display.max_columns', 500)
warnings.filterwarnings(action='ignore')
%matplotlib inline

## 2.2 Reading Data
### 2.2.1 Boston Airbnb Data

In [ ]:
# reading boston data
df_bos_lis = pd.read_csv('BostonData/listings.csv')
df_bos_rev = pd.read_csv('BostonData/reviews.csv')
df_bos_cal = pd.read_csv('BostonData/calendar.csv')

### 2.2.2 Seattle Airbnb Data

In [ ]:
# reading seattle data
df_sea_lis = pd.read_csv('SeattleData/listings.csv')
df_sea_rev = pd.read_csv('SeattleData/reviews.csv')
df_sea_cal = pd.read_csv('SeattleData/calendar.csv')

### 2.2.3 First View

In [ ]:
display(df_bos_cal.head(), df_sea_cal.head())

In [ ]:
display(df_bos_lis.head(1), df_sea_lis.head(1))

In [ ]:
display(df_bos_rev.head(), df_sea_rev.head())

# 3. Data Preparation

## 3.1 Reviews Dataset
My objective with this dataset is to classify each review as good or bad. Then, I'll count how many good and bad reviews each listing has. I believe this will be a good parameter for predicting both price and review rating. 

In [ ]:
# checking first rows of the dataset
df_bos_rev.head()

In [ ]:
# making a copy of the original dataset
df_bos_rev_c = df_bos_rev.copy()

# checking the dataset info
df_bos_rev_c.info()

### 3.1.1 Reviews classification process
1. I'll will create two lists, one with positive words and other with negative words. 
2. I'll count how many words of each list are present in each comment
3. The category wich has more words, will be the category of the review

In [ ]:
# good words list
positive_words = ['good', 'great', 'amazing', 'perfect', 'nice', 'cool', 'cozy', 'amazing', 'comfortable', 'loved',
                  'enjoyed', 'lovely', 'wonderful', 'fantastic', 'pleasure', 'brilliant', 'pleasant', 'superb',
                  'charming', 'awesome', 'beautiful', 'fun', 'excellent']

# bad words list
negative_words = ['bad', 'terrible', 'horrible', 'uncomfortable', 'dirty', 'cancelled', 'unconvenient', 'never',
                  'hated', 'disliked', 'ugly', 'boring', 'refund', 'exhausted', 'tired','garbage','not acceptable',
                  'refund', 'awful', 'damages', 'never', 'frustrated', 'canceled', 'cancel', 'panic', 'horror',
                  'worst']

# classification function
def classify_good_or_bad(comment):
    # making sure comment is string
    comment = str(comment)
    
    # removing all sorts of punctuation
    for char in string.punctuation:
        comment.replace(char, '')
    
    # make it all lowercase
    comment = comment.lower()
    
    # get all words
    comment = comment.split()
    
    # good words count
    positive_count = 0
    for word in comment:
        if word in positive_words:
            positive_count += 1
            
    # bad words count
    negative_count = 0
    for word in comment:
        if word in negative_words:
            negative_count += 1
            
    # classifying 
    if positive_count > negative_count:
        return 'positive'
    elif positive_count < negative_count:
        return 'negative'
    else:
        return 'unknwon'

### 3.1.2 Creating new good or bad review column

In [ ]:
# applying the classification function
df_bos_rev_c['review_cat'] = df_bos_rev_c.comments.apply(classify_good_or_bad)
df_bos_rev_c['review_cat'].value_counts()

### 3.1.3 Dropping unnecessary columns and creating dummy variables

In [ ]:
# pulling only the necessary columns
df_bos_rev_c = df_bos_rev_c[['listing_id', 'review_cat']]

# creating dummy variables for the good, bad or unknown reviews 
df_bos_rev_c = pd.concat([df_bos_rev_c['listing_id'], pd.get_dummies(df_bos_rev_c['review_cat'], prefix='review')], axis=1)
df_bos_rev_c.head()

### 3.1.4 Creating the boston reviews dataframe

In [ ]:
# creating the boston reviews dataframe by grouping the amount of positive rows by listing_id
boston_reviews = df_bos_rev_c.groupby(['listing_id']).sum().reset_index()
boston_reviews.head()

**This looks great! Now let's apply the same steps in the Seattle dataframe and merge them!**

### 3.1.5 Applying the same steps on the Seattle dataframe

In [ ]:
# copying the dataframe
df_sea_rev_c = df_sea_rev.copy()

# applying the classification function
df_sea_rev_c['review_cat'] = df_sea_rev_c.comments.apply(classify_good_or_bad)

# pulling only the necessary columns
df_sea_rev_c = df_sea_rev_c[['listing_id', 'review_cat']]

# creating dummy variables for the good, bad or unknown reviews 
df_sea_rev_c = pd.concat([df_sea_rev_c['listing_id'], pd.get_dummies(df_sea_rev_c['review_cat'], prefix='review')], axis=1)

# creating the boston reviews dataframe by grouping the amount of positive rows by listing_id
seattle_reviews = df_sea_rev_c.groupby(['listing_id']).sum().reset_index()

# checking if everything went ok
seattle_reviews.head()

### 3.1.6 Combining Seattle and Boston dataframes into the reviews dataframe

In [ ]:
reviews = pd.concat([boston_reviews, seattle_reviews], axis=0)
reviews.head()

## 3.2 Calendar Dataset
My objective with this dataset is to extract the percentage of the year that each listing was occupied. That way I will be able to explore how the listings features make them more or less attractive to travelers.

In [ ]:
# creating a copy of the dataset
df_bos_cal_c = df_bos_cal.copy()

# checking dataset info
df_bos_cal_c.info()

### 3.2.1 Pulling only necessary columns

In [ ]:
# pulling only the listing and availble columns 
df_bos_cal_c = df_bos_cal_c[['listing_id', 'available']]
df_bos_cal_c.head()

### 3.2.2 Formatting the 'available' column

In [ ]:
df_bos_cal_c.available.value_counts()

In [ ]:
# applying lambda function to make true of false
df_bos_cal_c.available = df_bos_cal_c.available.apply(lambda x: x == 't')
df_bos_cal_c.head()

In [ ]:
df_bos_cal_c.available.value_counts()

### 3.2.3 Getting the Dummies for the 'available' column

In [ ]:
df_bos_cal_c = pd.concat([df_bos_cal_c.listing_id, pd.get_dummies(df_bos_cal_c.available, prefix='available')], axis=1)
df_bos_cal_c.head()

### 3.2.4 Creating the Boston Calendar dataframe

In [ ]:
# grouping the amount of occupied and taken days for each listing
boston_calendar = df_bos_cal_c.groupby('listing_id').sum().reset_index()
boston_calendar.head()

In [ ]:
# creating the occupation_rate column: represents the % of the year that the listing is occupied
boston_calendar['occupation_rate'] = boston_calendar['available_False'] / 365

# removing unnecessary columns
boston_calendar = boston_calendar[['listing_id', 'occupation_rate']]
boston_calendar.head()

That looks **amazing**! Now let's do the same with the seattle dataset!

### 3.2.5 Applying the same steps on the seattle dataframe

In [ ]:
# creating a copy of the dataset
df_sea_cal_c = df_sea_cal.copy()

# pulling only the listing and availble columns 
df_sea_cal_c = df_sea_cal_c[['listing_id', 'available']]

# applying lambda function to make true of false
df_sea_cal_c.available = df_sea_cal_c.available.apply(lambda x: x == 't')

# getting the dummies
df_sea_cal_c = pd.concat([df_sea_cal_c.listing_id, pd.get_dummies(df_sea_cal_c.available, prefix='available')], axis=1)

# grouping the amount of occupied and taken days for each listing
seattle_calendar = df_sea_cal_c.groupby('listing_id').sum().reset_index()

# creating the occupation_rate column: represents the % of the year that the listing is occupied
seattle_calendar['occupation_rate'] = seattle_calendar['available_False'] / 365

# removing unnecessary columns
seattle_calendar = seattle_calendar[['listing_id', 'occupation_rate']]
seattle_calendar.head()

### 3.2.6 Combining Boston and Seattle Dataframes

In [ ]:
calendar = pd.concat([boston_calendar, seattle_calendar])
calendar.head()

## 3.3 Listings Dataset
My objective with this dataset is to get the main features of each listing and it's review status. That way, I'll be able to run a model to predict the listing price and the review rate.

In [ ]:
# creating copy of the dataset
df_bos_lis_c = df_bos_lis.copy()

### 3.3.1 Keeping only necessary columns

In [ ]:
# selecting only necessary columns
df_bos_lis_c = df_bos_lis_c[['id', 'market', 'host_is_superhost', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 
                             'bed_type', 'price', 'cleaning_fee', 'number_of_reviews', 'review_scores_rating', 'cancellation_policy']]

df_bos_lis_c.head(3)

### 3.3.2 Checking data types

In [ ]:
# checking info of the dataset
df_bos_lis_c.info()

Let's check the 'object' type columns to see if any of them should be a numerical one

In [ ]:
# pulling only the 'object' type column 
df_bos_lis_c.select_dtypes(include=['object']).head(1)

Ok, so there are 2 columns that should be numerical: 'price' and 'cleaning_fee'. **Let's fix them:**

In [ ]:
# applying the functions
df_bos_lis_c.price = df_bos_lis_c.price.apply(lambda x: x.replace('$', ''))
df_bos_lis_c.price = df_bos_lis_c.price.apply(lambda x: x.replace(',', '.'))
df_bos_lis_c.price = df_bos_lis_c.price.apply(lambda x: x[:-3])

df_bos_lis_c.cleaning_fee = df_bos_lis_c.cleaning_fee.apply(lambda x: str(x) if x else None)
df_bos_lis_c.cleaning_fee = df_bos_lis_c.cleaning_fee.apply(lambda x: x.replace('$', '') if x else None)
df_bos_lis_c.cleaning_fee = df_bos_lis_c.cleaning_fee.apply(lambda x: x.replace(',', '.') if x else None)
df_bos_lis_c.cleaning_fee = df_bos_lis_c.cleaning_fee.apply(lambda x: x[:-3] if x else None)
df_bos_lis_c.cleaning_fee = df_bos_lis_c.cleaning_fee.apply(lambda x: np.nan if x=='' else x)

# checking columns
df_bos_lis_c[['price', 'cleaning_fee']].head(3)

**OK**, they look alright. Now I can fix the datatypes!

In [ ]:
# Changing the datatypes 
df_bos_lis_c[['price', 'cleaning_fee']] = df_bos_lis_c[['price', 'cleaning_fee']].astype('float64')
df_bos_lis_c[['price', 'cleaning_fee']].info()

### 3.3.3 Dealing with null values

In [ ]:
# checking the amount of nulls in each column
df_bos_lis_c.isnull().sum()

### Handling nulls in categorical columns:

In [ ]:
# checking the amount of nulls in each object column  
df_bos_lis_c.select_dtypes(include=['object']).isnull().sum()

#### How am I approaching this:
* **Market**: all of them should be 'Boston' so I'll just put that 
* **Property_type**: Since it's only 3, I'll fill them with the mode

In [ ]:
# filling market nulls
df_bos_lis_c.market = df_bos_lis_c.market.fillna('Boston')

# filling property_type nulls
df_bos_lis_c.property_type = df_bos_lis_c.property_type.fillna(df_bos_lis_c.property_type.mode().values[0])

# making sure there are no more nulls
df_bos_lis_c.select_dtypes(include=['object']).isnull().sum()

### Handling nulls in numerical columns:

In [ ]:
# checking the amount of nulls in each numerical column
df_bos_lis_c.select_dtypes(include=['int64', 'float64']).isnull().sum()

#### How am I approaching this:

**bathrooms, bedrooms and beds**: Since there are only a handfull of these missing and calculating their mean doesn't make a lot of sense, I'll fill the null values with the mode.


In [ ]:
# filling bathrooms null values with the mode
df_bos_lis_c['bathrooms'] = df_bos_lis_c['bathrooms'].fillna(df_bos_lis_c['bathrooms'].mode().values[0])

# filling bedrooms null values with the mode
df_bos_lis_c['bedrooms'] = df_bos_lis_c['bedrooms'].fillna(df_bos_lis_c['bedrooms'].mode().values[0])

# filling beds null values with the mode
df_bos_lis_c['beds'] = df_bos_lis_c['beds'].fillna(df_bos_lis_c['beds'].mode().values[0])

# making sure it worked
df_bos_lis_c[['bathrooms', 'bedrooms', 'beds']].isnull().sum()

**cleaning_fee**: By looking at the below chart it get's clear that it's better to fill the null values with the mean =~ 68. 

In [ ]:
sns.displot(df_bos_lis_c['cleaning_fee'], bins=10)
plt.title('cleaning_fee distribution', size=13)
plt.show()

In [ ]:
# filling the null values with the mean 
df_bos_lis_c['cleaning_fee'] = df_bos_lis_c['cleaning_fee'].fillna(df_bos_lis_c['cleaning_fee'].mean())

# checking if it worked
df_bos_lis_c['cleaning_fee'].isnull().sum()

### 3.3.4 Dealing with Categorical Features

In [ ]:
df_bos_lis_c.select_dtypes(include='object').head()

**market**: I'll just create a 'boston' column that contains 1 for all values in boston

In [ ]:
# creates new column based on market column
df_bos_lis_c['boston'] = np.where(df_bos_lis_c['market']=='Boston', 1, 0)
df_bos_lis_c['seattle'] = np.where(df_bos_lis_c['market']=='Seattle', 1, 0)

# drops olde market column
df_bos_lis_c.drop('market', axis=1, inplace=True)

# shows boston column
df_bos_lis_c[['boston', 'seattle']].head()

**host_is_superhost**: I'll again just create a new column 'superhost' that contains 1 if true else 0

In [ ]:
# creates new column superhost based on host_is_superhost column
df_bos_lis_c['superhost'] = np.where(df_bos_lis_c['host_is_superhost']=='t', 1, 0)

# drop the 'host_is_superhost' column
df_bos_lis_c.drop('host_is_superhost', axis=1, inplace=True)

# shows superhost column
df_bos_lis_c['superhost'].head()


**property_type**: I'll get the dummies for this column

In [ ]:
df_bos_lis_c = pd.concat([df_bos_lis_c.drop('property_type',axis=1), pd.get_dummies(df_bos_lis_c.property_type)], axis=1)
df_bos_lis_c.head()

**room_type**: I'll also just get the dummies for this one

In [ ]:
df_bos_lis_c = pd.concat([df_bos_lis_c.drop('room_type',axis=1), pd.get_dummies(df_bos_lis_c.room_type)], axis=1)
df_bos_lis_c.head()

**bed_type**: Again, just the dummies

In [ ]:
df_bos_lis_c = pd.concat([df_bos_lis_c.drop('bed_type', axis=1), pd.get_dummies(df_bos_lis_c.bed_type)], axis=1)
df_bos_lis_c.head()

**cancellation_policy**: Just the dummies

In [ ]:
df_bos_lis_c = pd.concat([df_bos_lis_c.drop('cancellation_policy', axis=1), pd.get_dummies(df_bos_lis_c.cancellation_policy)], axis=1)
df_bos_lis_c.head()

Rounding the 'cleaning_fee' column

In [ ]:
# applying the round function
df_bos_lis_c['cleaning_fee'] = df_bos_lis_c['cleaning_fee'].round(decimals = 2)

Creating the boston listings dataset

In [ ]:
boston_listings = df_bos_lis_c
boston_listings.head()

### 3.3.5 Applying the same steps to the Seattle dataframe

In [ ]:
# creating copy of the dataset
df_sea_lis_c = df_sea_lis.copy()

# selecting only necessary columns
df_sea_lis_c = df_sea_lis_c[['id', 'market', 'host_is_superhost', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 
                             'bed_type', 'price', 'cleaning_fee', 'number_of_reviews', 'review_scores_rating', 'cancellation_policy']]

# applying the functions
df_sea_lis_c.price = df_sea_lis_c.price.apply(lambda x: x.replace('$', ''))
df_sea_lis_c.price = df_sea_lis_c.price.apply(lambda x: x.replace(',', '.'))
df_sea_lis_c.price = df_sea_lis_c.price.apply(lambda x: x[:-3])

df_sea_lis_c.cleaning_fee = df_sea_lis_c.cleaning_fee.apply(lambda x: str(x) if x else None)
df_sea_lis_c.cleaning_fee = df_sea_lis_c.cleaning_fee.apply(lambda x: x.replace('$', '') if x else None)
df_sea_lis_c.cleaning_fee = df_sea_lis_c.cleaning_fee.apply(lambda x: x.replace(',', '.') if x else None)
df_sea_lis_c.cleaning_fee = df_sea_lis_c.cleaning_fee.apply(lambda x: x[:-3] if x else None)
df_sea_lis_c.cleaning_fee = df_sea_lis_c.cleaning_fee.apply(lambda x: np.nan if x=='' else x)

# Changing the datatypes 
df_sea_lis_c[['price', 'cleaning_fee']] = df_sea_lis_c[['price', 'cleaning_fee']].astype('float64')

# filling market nulls
df_sea_lis_c.market = df_sea_lis_c.market.fillna('Boston')

# filling property_type nulls
df_sea_lis_c.property_type = df_sea_lis_c.property_type.fillna(df_sea_lis_c.property_type.mode().values[0])

# filling bathrooms null values with the mode
df_sea_lis_c['bathrooms'] = df_sea_lis_c['bathrooms'].fillna(df_sea_lis_c['bathrooms'].mode().values[0])

# filling bedrooms null values with the mode
df_sea_lis_c['bedrooms'] = df_sea_lis_c['bedrooms'].fillna(df_sea_lis_c['bedrooms'].mode().values[0])

# filling beds null values with the mode
df_sea_lis_c['beds'] = df_sea_lis_c['beds'].fillna(df_sea_lis_c['beds'].mode().values[0])

# filling the null values with the mean 
df_sea_lis_c['cleaning_fee'] = df_sea_lis_c['cleaning_fee'].fillna(df_sea_lis_c['cleaning_fee'].mean())

# creates new column based on market column
df_sea_lis_c['boston'] = np.where(df_sea_lis_c['market']=='Boston', 1, 0)
df_sea_lis_c['seattle'] = np.where(df_sea_lis_c['market']=='Seattle', 1, 0)

# drops olde market column
df_sea_lis_c.drop('market', axis=1, inplace=True)

# creates new column superhost based on host_is_superhost column
df_sea_lis_c['superhost'] = np.where(df_sea_lis_c['host_is_superhost']=='t', 1, 0)

# drop the 'host_is_superhost' column
df_sea_lis_c.drop('host_is_superhost', axis=1, inplace=True)

# getting the dummies
df_sea_lis_c = pd.concat([df_sea_lis_c.drop('property_type',axis=1), pd.get_dummies(df_sea_lis_c.property_type)], axis=1)
df_sea_lis_c = pd.concat([df_sea_lis_c.drop('room_type',axis=1), pd.get_dummies(df_sea_lis_c.room_type)], axis=1)
df_sea_lis_c = pd.concat([df_sea_lis_c.drop('bed_type', axis=1), pd.get_dummies(df_sea_lis_c.bed_type)], axis=1)
df_sea_lis_c = pd.concat([df_sea_lis_c.drop('cancellation_policy', axis=1), pd.get_dummies(df_sea_lis_c.cancellation_policy)], axis=1)

# applying the round function
df_sea_lis_c['cleaning_fee'] = df_sea_lis_c['cleaning_fee'].round(decimals = 2)

# creating the seattle listings dataset
seattle_listings = df_sea_lis_c

### 3.3.6 Conbining seattle and boston datasets

In [ ]:
display(seattle_listings.head(1), boston_listings.head(1))

In [ ]:
listings = pd.concat([seattle_listings, boston_listings])
listings.head()

Great. As you can see, there were a few columns in the boston dataset there weren't in the seattle one. As consequence, thoso columns are completely NaN in the final dataset. Thankfully those are all binary columns, so I can just fill the up with 0.

### 3.3.7 Fixing the Listings dataset

In [ ]:
listings.isnull().sum()

In [ ]:
listings[['Bungalow', 'Cabin', 'Chalet', 'Tent', 'Treehouse', 'Yurt', 'Entire Floor', 
          'Guesthouse', 'Villa', 'super_strict_30']] = listings[['Bungalow', 'Cabin', 'Chalet', 'Tent', 'Treehouse', 
                                                                 'Yurt', 'Entire Floor', 'Guesthouse', 'Villa', 'super_strict_30']].fillna(0)

listings.isnull().sum()

That worked!

### 3.3.8 Fixing the review_scores_rating column

In [ ]:
listings[(listings.review_scores_rating.isnull() == True) & (listings.number_of_reviews != 0)]

Ok, there are 2 different ocasions where the review_scores_rating is NaN
1. The number of reviews is 0
    - In this case, It makes sense to keep the rows and create a new column telling the the value is null
2. The number of reviews is greater then 0
    - In this case it doesn't make sense, so I'll just drop these rows

In [ ]:
# dropping rows with reviews and without review score
listings = listings.drop(listings[(listings['review_scores_rating'].isnull()) & (listings['number_of_reviews']>0)].index)

# checking if worked
listings[(listings['review_scores_rating'].isnull()) & (listings['number_of_reviews']>0)]

Great, it worked!

#### Creating a column to showcase were the reviews are missing 

In [ ]:
listings['missing_review'] = np.where(listings['review_scores_rating'].isnull(), 1, 0)
listings[['missing_review', 'review_scores_rating']][listings['missing_review']==1].head()

#### Filling review_scores_rating missing values with 0

In [ ]:
listings['review_scores_rating'] = listings['review_scores_rating'].fillna(0)
listings.isnull().sum()

**Perfect!!**

### 3.4 Combining all three datasets

In [ ]:
cal_rev = pd.merge(calendar, reviews, left_on='listing_id', right_on='listing_id', how='left')
cal_rev.head()

In [ ]:
df = pd.merge(cal_rev, listings, left_on='listing_id', right_on='id', how='left')
df.head()

In [ ]:
df.isnull().sum()

Ok, we have a few problems here. Again!

### 3.4.1 Cleaning the final dataframe

**problem 1**: There are a few (150) listings that weren't in the listings dataset

In [ ]:
df[df['id'].isnull()]

**My solution**: I'll need to drop these rows since it will be impossible to fill the these null values 

In [ ]:
df = df.drop(df[df['id'].isnull()].index)
df[df['id'].isnull()]

Ok, that worked!

**problem 2:** All the 1374 listings that doesn't have any reviews have null is the number of positive and negative reviews

In [ ]:
df[(df['review_negative'].isnull()) & (df['review_positive'].isnull()) & (df['review_unknwon'].isnull())].iloc[:, [2,3,4,12]]

**my solution:** I'll just add 0 for all those values

In [ ]:
df = df.fillna(0)
df.isnull().sum().sum()

Great, new there aren't any null values in the dataset!

### 3.4.2 Fixing the final df datatypes

There are a bunch of columns that should be binary and are as float64 dataypes. I'll just change there datatypes to uint64

In [ ]:
df.head()

In [ ]:
# changing the first columns datatypes
df.iloc[:, [2, 3, 4, 5, 6, 9, 12]] = df.iloc[:, [2, 3, 4, 5, 6, 9, 12]].astype('uint64')

# changing the dataype of the rest of the columns
df.iloc[:, 14:] = df.iloc[:, 14:].astype('uint64')

# 
df.head()

### 3.4.3 Dropping the unnecessary columns

In this case, the listing_id and the id columns because they won't be useful for the modeling:

In [ ]:
df.drop(['listing_id', 'id'], axis=1, inplace=True)
df.head()

**That looks great!** Now I can go for the modeling!

### 3.5 Analysing the Data

In order to facilitate my analysis, I'll need to undo some of the changes I previously did. For that, I'll just create a new dataset.

In [ ]:
# creating new analysis df
analysis_df = df.copy() 

# making a few changes to facilitate the analysis 

# creating categorical city column
analysis_df['city'] = np.where(analysis_df.boston==1, 'boston', 'seattle')


# creating categorical occupation column
def classify_occupation(value: float) -> str:
    """Function to classify the occupation rate into categories, such as: 
    - always occupied
    - often occupied 
    - sometimes occupied 
    - rarely occupied

    Args:
        value (float): the occupation rate of the propery

    Returns:
        str: the category in which the property fits
    """
    if value >= 0.95:
        return 'always occupied'
    if value >= 0.75:
        return 'often occupied'
    if value >= 0.50:
        return 'sometimes occupied'
    if value >= 0.15:
        return 'rarely occupied'
    return 'almost never occupied'



# creating categorical listing quality function
def classify_listing_quality(value: float) -> str:
    """Function to classify the listing quality into categories, such as:
    - amazing
    - very good
    - decent 
    - bad

    Args:
        value (float): Reviews rating for that listing

    Returns:
        str: category in which the property fits
    """
    if value >= 0.75:
        return 'good'
    return 'bad'


# applying functions to the dataset
analysis_df['occupation'] = analysis_df['occupation_rate'].apply(classify_occupation)
analysis_df['quality'] = analysis_df['review_scores_rating'].apply(classify_listing_quality)

# removing outliers
analysis_df = analysis_df.drop(analysis_df[analysis_df['occupation_rate']>1].index)

analysis_df.head()

### 3.5.1 Question 1: 

#### **Which city has the best listings? Which one has more expensive ones? Is there a connection in that?**

In [ ]:
# grouping data
df1 = analysis_df.groupby('city')['review_scores_rating'].mean().to_frame().reset_index()

# creating figure
plt.figure(figsize=(12,8))

# creating bar plot
ax = sns.barplot(x='city', y='review_scores_rating', data=df1, palette="Set1", edgecolor='black')

# personalizing bar plot
ax.bar_label(ax.containers[0], size=12);
ax.set_title('Average Review Rating by City', size=15)
ax.set_ylabel('Average Review Rating', size=12)
ax.set_xlabel('City', size=12)

# showing bar plot
plt.show()

In [ ]:
# grouping data
df2 = analysis_df.groupby('city')[['review_negative', 'review_positive']].sum()

# building plot
ax = df2.plot(kind='bar', stacked=False, color=['tomato', 'skyblue'], figsize=(12,8), edgecolor='black')

# personalizing plot
ax.set_title('Number of Positive and Negative Reviews by City', size=15)
ax.set_xlabel('City', size=15)
ax.set_ylabel('Number of Reviews', size=15)

for i in ax.containers:
    ax.bar_label(i, size=12)

plt.show()

In [ ]:
# grouping data
df3 = analysis_df.groupby('city')['price'].mean().to_frame().reset_index()
df3['price'] = df3['price'].apply(lambda x: round(x, 2))

# creating figure
plt.figure(figsize=(12,8))

# building plot
ax = sns.barplot(x='city', y='price', data=df3, palette='Set1', edgecolor='black')

# personalizing plot
ax.set_title('Average price by City', size=15)
ax.set_ylabel('Average Price', size=15)
ax.set_xlabel('City', size=15)
ax.bar_label(ax.containers[0], size=12)

# showing plot
plt.show()

### My Results: 
- After looking at the data, it got clear that the city of **Seattle has the best listings**. It not only has a **higher average review rating**, 78.8 compared with 72.1 in Boston, but it also has a much higher positive/negative review ratio, 55.4 compared with 31.9 in Boston. In addition to that, **Boston has a much higher average price**, 168.70 dollars compared with 127.02 dollars in Seattle. That means that more expensive houses are not necessarily better. 
- In my opinion, when people get into more expensive houses they tend to be more demanding. In that case, sometimes it leads to people geting disapointed with the listing, leading to negative reviews. 

### 3.5.2 Question 2:

### What's the connection between price and occupation rate? Where does the quality of the listing fits in that?

In [ ]:
# grouping data
df4 = analysis_df.groupby('occupation')['price'].mean().to_frame().reset_index().sort_values(by='price')

# plotting 
plt.figure(figsize=(12,8))
ax = sns.barplot(data=df4, x='occupation', y='price', edgecolor='black', palette='Set1')
ax.set_xlabel('Occupation Rate', size=15)
ax.set_ylabel('Average Price', size=15)
ax.set_title('Average Price by Occupation Rate', size=15)

for i in ax.containers:
    ax.bar_label(i, size=12)
plt.show()

#### Plotting the relationshiop between price and occupation_rate

In [ ]:
plt.figure(figsize=(12,8))
ax = sns.scatterplot(data=analysis_df, y='price', x='occupation_rate', hue='quality', palette='Set1')
ax.set_xlabel('Occupation Rate', size=15)
ax.set_ylabel('Price', size=15)
ax.set_title('Price x Occupation Rate', size=15)
plt.show()

### My Results:
- The correlation between price and occupation rate is not very strong. That means that lowering the price of your house won't necessarily call for more attention.
- One interesting finding is that expensive houses (above 400 dollars) tend to have either a very high or very low occupation rate. 

### 3.5.3 Question 3:

### What are the main features that influences the review rates? What about the prices?

In order to answer that question, I'll need to build some machine learning models. That leads us to the next section:

# 4. Data Modelling

## 4.1 Predicting the review scores:

In order to better predict the review scores, I'll first drop the 0 filled review_scores rating rows. That way our model won't be biased. 

In [ ]:
# dropping rows
df_review = df.drop(df[df['review_scores_rating']==0].index)

# checking dataframe
df_review.head()

**Splitting into train and test datasets**

In [ ]:
# regressors
X = df_review.drop(columns=['review_scores_rating', 'missing_review'], axis=1)

# target
y = df_review['review_scores_rating']

# train and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

X_train.shape, X_test.shape

**Function to automate model creation, training, and evaluation**

In [ ]:
def build_model(ml_model, X_train, X_test, y_train, y_test, evaluation_methods: list):
    """Function to automate the model creation, training, prediction, and evaluation.

    Args:
        ml_model (object): Algorithm that will be built
        X_train (Array): training regressors
        X_test (Array): testing regressors
        y_train (Array): training target
        y_test (Array): testing target
        evaluation_methods (list): evaluation methods that will be used to see the model performace
    """
    # creates model
    model = ml_model
    
    # trains model
    model.fit(X_train, y_train)
    
    # predicts model
    y_pred = model.predict(X_test)
    
    # evaluates model
    evaluation_results = []
    
    for method in evaluation_methods:
        evaluation_results.append(method(y_test, y_pred))
        
    # prints results
    for method, result in zip(evaluation_methods, evaluation_results):
        print(f'{method}: {result}')

**Building Linear Regression Model**

In [ ]:
build_model(LinearRegression(normalize=True), X_train, X_test, y_train, y_test, [r2_score, mean_squared_error])

**Building Ridge Regression Model**

In [ ]:
build_model(RidgeCV(), X_train, X_test, y_train, y_test, [r2_score, mean_squared_error])

**Building Lasso Regression Model**

In [ ]:
build_model(LassoCV(), X_train, X_test, y_train, y_test, [r2_score, mean_squared_error])

**Building Elastic Net Model**

In [ ]:
build_model(ElasticNetCV(), X_train, X_test, y_train, y_test, [r2_score, mean_squared_error])

**Building Random Forest Model**

In [ ]:
build_model(RandomForestRegressor(), X_train, X_test, y_train, y_test, [r2_score, mean_squared_error])

**Since the Random Forest was the best model, let's build it again and tune the hyperparameters:**

**Before hyperparameter tuning:**

In [ ]:
# building model 
rf = RandomForestRegressor()

# training model
rf.fit(X_train, y_train)

# predicting with the model
y_pred = rf.predict(X_test)

# evaluating the model
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f'R2 Score: {r2},\nMean Squared Error: {mse}')

**After hyperparameter tuning:**
- Using RandomizedSearchCV

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
# instantiate model
rf = RandomForestRegressor()

# searching for parameters
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# train model
rf_random.fit(X_train, y_train)

Taking a look on the best parameters

In [ ]:
rf_random.best_params_

Creating the tuned model

In [ ]:
# instantiating the model
tuned_rf = rf_random.best_estimator_

# training the model
tuned_rf.fit(X_train, y_train)

# predicting with the model
y_pred_tuned = tuned_rf.predict(X_test)

# evaluating the model
r2 = r2_score(y_test, y_pred_tuned)
mse = mean_squared_error(y_test, y_pred_tuned)

print(f'R2 Score: {r2},\nMean Squared Error: {mse}')


Ok, that model isn't much better, but it has improved. Let's check the features importances:

In [ ]:
feature_importances = pd.DataFrame(tuned_rf.feature_importances_, index = X_train.columns,columns=['importance']).sort_values('importance',ascending=False)
feature_importances

### My Results:

- The most important features when predicting an Airbnb listing review score is the price followed by the occupation rate and the number of good and bad reviews. **That actually makes a lot of sense**. 
    - People will evaluate your house based on the price you put on it. If you charge too much, people will probably be disappointed and review it poorly. 
- Other important indicator is the **cleaning fee**.
    - Apperently people dont' like to spend a lot when it's time to clean. Quite interesting. 

## 4.2 Predicting the Price:

In [ ]:
df.head()

Let's separate the training and testing datasets

In [ ]:
# regressors
X = df.drop(['price'], axis=1)
y = df['price']

# train and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)

print(X_train.shape)
print(X_test.shape)

Let's build the models again:

**Linear Regression**

In [ ]:
build_model(LinearRegression(), X_train, X_test, y_train, y_test, [r2_score, mean_squared_error])

**Lasso Regression**

In [ ]:
build_model(LassoCV(), X_train, X_test, y_train, y_test, [r2_score, mean_squared_error])

**Ridge Regression**

In [ ]:
build_model(LassoCV(), X_train, X_test, y_train, y_test, [r2_score, mean_squared_error])

**ElasticNet**

In [ ]:
build_model(ElasticNetCV(), X_train, X_test, y_train, y_test, [r2_score, mean_squared_error])

In [ ]:
# using the previously created function
build_model(RandomForestRegressor(), X_train, X_test, y_train, y_test, [r2_score, mean_squared_error])

Once again, the Random Forest Model was the best. Let's build it again and tune it!

**Before hyperparameter tuning**

In [ ]:
# building model 
rf_price = RandomForestRegressor()

# training model
rf_price.fit(X_train, y_train)

# predicting with the model
y_pred = rf_price.predict(X_test)

# evaluating the model
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f'R2 Score: {r2},\nMean Squared Error: {mse}')

**After hyperparameter tuning**

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
# instantiate model
rf_price_tuned = RandomForestRegressor()

# searching for parameters
rf_price_tuned = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# train model
rf_price_tuned.fit(X_train, y_train)

In [ ]:
# instantiating the model
tuned_rf = rf_random.best_estimator_

# training the model
tuned_rf.fit(X_train, y_train)

# predicting with the model
y_pred_tuned = tuned_rf.predict(X_test)

# evaluating the model
r2 = r2_score(y_test, y_pred_tuned)
mse = mean_squared_error(y_test, y_pred_tuned)

print(f'R2 Score: {r2},\nMean Squared Error: {mse}')

In [ ]:
feature_importances = pd.DataFrame(tuned_rf.feature_importances_, index = X_train.columns,columns=['importance']).sort_values('importance',ascending=False)
feature_importances

### My Results:
- The most important features when predicting the price of a listing are:
    1. The **cleaning fee**: When it's very expensive to clean a house, the house must be very expensive as well
    2. The **number of bedrooms**: More bedrooms -> bigger house -> more expensive house. Makes Sense.
    3. The **number of people it accommodates**: More people -> more bedrooms -> bigger house -> more expensive house. Makes Sense.

# 5 Results & Evaluation

## Which city has the best listings? Which one has more expensive ones? Is there a conection in that?

- After looking at the data, it got clear that the city of **Seattle has the best listings**. It not only has a **higher average review rating**, 78.8 compared with 72.1 in Boston, but it also has a much higher positive/negative review ratio, 55.4 compared with 31.9 in Boston. In addition to that, **Boston has a much higher average price**, 168.70 dollars compared with 127.02 dollars in Seattle. That means that more expensive houses are not necessarily better. 
- In my opinion, when people get into more expensive houses they tend to be more demanding. In that case, sometimes it leads to people geting disapointed with the listing, leading to negative reviews. 

## What's the connection between price and occupation rate? Where does the quality of the listing fits in that?

- The correlation between price and occupation rate is not very strong. That means that lowering the price of your house won't necessarily call for more attention.
- One interesting finding is that expensive houses (above 400 dollars) tend to have either a very high or very low occupation rate. 

## What are the main features that influence the review rates? What about the prices?

### Review Rates:
- The most important features when predicting an Airbnb listing review score is the price followed by the occupation rate and the number of good and bad reviews. **That actually makes a lot of sense**. 
    - People will evaluate your house based on the price you put on it. If you charge too much, people will probably be disappointed and review it poorly. 
- Other important indicator is the **cleaning fee**.
    - Apperently people dont' like to spend a lot when it's time to clean. Quite interesting. 
    
### Listing Prices:
- The most important features when predicting the price of a listing are:
    1. The **cleaning fee**: When it's very expensive to clean a house, the house must be very expensive as well
    2. The **number of bedrooms**: More bedrooms -> bigger house -> more expensive house. Makes Sense.
    3. The **number of people it accommodates**: More people -> more bedrooms -> bigger house -> more expensive house. Makes Sense.